In [3]:
import tkinter
import os
import tkinter.messagebox
import customtkinter
import pandas as pd
import unicodedata
import requests
from tkinter import filedialog
import tempfile
from PIL import Image, ImageTk
from tkintertable.Tables import TableCanvas

# Definindo a aparência padrão como "light"
customtkinter.set_appearance_mode("light")
customtkinter.set_default_color_theme("green")

class AppSaudados(customtkinter.CTk):
    def __init__(self):
        super().__init__()

        # Definindo a aparência padrão como "light"
        customtkinter.set_appearance_mode("light")
        customtkinter.set_default_color_theme("green")

        self.title("SAUDADOS")
        self.geometry(f"{1100}x{580}")
        self.resizable(width=False, height=False)

        self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure((1, 2), weight=0)
        self.grid_rowconfigure((0, 1, 2), weight=1)

        self.sidebar_frame = customtkinter.CTkFrame(self, width=140, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=10, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)

        #titulo
        self.logo_label = customtkinter.CTkLabel(self.sidebar_frame, text="SAUDADOS – CIÊNCIA DE DADOS E SAÚDE", font=customtkinter.CTkFont(size=12, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        #entrada
        self.Entrada_label = customtkinter.CTkLabel(self.sidebar_frame, text="Olá, Bem Vindo!", font=customtkinter.CTkFont(size=10, weight="bold"))
        self.Entrada_label.grid(row=1, column=0, padx=20, pady=(10))

        texto = "O objetivo desse sistema é apoiar as ações de pesquisa\n\n" +"utilizando dados de bases da saúde, bem como dados\n\n" +"demográficos entre outros.\n\n" + "Instruções:\n\n"
        
        # Usando Label para exibir o texto
        self.labelTexto = customtkinter.CTkLabel(self.sidebar_frame, text=texto, justify="left", font=customtkinter.CTkFont(size=10))
        self.labelTexto.grid(row=2, column=0, padx=(20, 0), pady=(20, 0), sticky="nsew")

        # Cria um arquivo temporário para armazenar temporariamente o DataFrame
        self.temp_file = tempfile.NamedTemporaryFile(delete=False)

        self.dataframes_carregados = []

        # Adicione esta linha para inicializar a variável de cancelamento
        self._cancel_load = False
        

        #mudar tema
        self.appearance_mode_label = customtkinter.CTkLabel(self.sidebar_frame, text="Aparência:", anchor="w")
        self.appearance_mode_label.grid(row=5, column=0, padx=20, pady=(10, 0))
        self.appearance_mode_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["Light", "Dark", "System"],
                                                                       command=self.change_appearance_mode_event)
        self.appearance_mode_optionemenu.grid(row=6, column=0, padx=20, pady=(10, 10))
        self.scaling_label = customtkinter.CTkLabel(self.sidebar_frame, text="Escala:", anchor="w")
        self.scaling_label.grid(row=7, column=0, padx=20, pady=(10, 0))
        self.scaling_optionemenu = customtkinter.CTkOptionMenu(self.sidebar_frame, values=["80%", "90%", "100%", "110%", "120%"],
                                                               command=self.change_scaling_event)
        self.scaling_optionemenu.grid(row=8, column=0, padx=20, pady=(10, 20))
        
        self.selected_file = None  # Armazenar o arquivo selecionado
        self.selected_extension = None  # Armazenar o tipo de extensão selecionado

        #fundo
        self.imagem_label = customtkinter.CTkFrame(self, width=200, corner_radius=0)
        self.imagem_label.grid(row=0, column=1, rowspan=5, padx=(10, 0), pady=(0, 0), sticky="nsew")
        self.imagem_label.grid_rowconfigure(0, weight=1)

    #    # Carregue a imagem usando ImageTk.PhotoImage
    #     self.ctk_image = ImageTk.PhotoImage(Image.open("./fundoSAUDADOS.jpg"))
    #     self.image_label = customtkinter.CTkLabel(self.imagem_label, image=self.ctk_image)
    #     self.image_label.grid(row=0, column=1, rowspan=5, sticky="nsew")

    #     # Atualize a imagem
    #     self.image_label.configure(image=self.ctk_image)

        #base de dados
        self.label_base_dados = customtkinter.CTkLabel(self.imagem_label, text="Selecione a Base de Dados:", anchor="w")
        self.label_base_dados.grid(row=0, column=0, padx=(20, 0), pady=(0, 0))

        # Obter todos os arquivos de um diretório específico
        diretorio = "D:/Area de Trabalho/Repositorys/PIBITI-IFBA-CNPq-2023-2024/ProjetoSaudados/BaseDadosTeste-Saudados"  
        arquivos = [arq for arq in os.listdir(diretorio) if os.path.isfile(os.path.join(diretorio, arq))]

        self.menu_base_dados = customtkinter.CTkOptionMenu(self.imagem_label, values=arquivos, command=self.on_base_dados_selected)
        self.menu_base_dados.grid(row=1, column=0, padx=(20, 0), pady=(5, 0))

        # EXTENSAO
        self.label_extensao = customtkinter.CTkLabel(self.imagem_label, text="Selecione a Extensão:", anchor="w")
        self.label_extensao.grid(row=2, column=0, padx=(20, 0), pady=(10, 0))
        self.menu_extensao = customtkinter.CTkOptionMenu(self.imagem_label, values=[".csv", ".xlsx", ".json", ".dbf", ".txt"], command=self.on_extensao_selected)
        self.menu_extensao.grid(row=3, column=0, padx=(20, 0), pady=(15, 0))

        #Botao ler arquivo
        self.button_LerArquivo = customtkinter.CTkButton(self.imagem_label, text="Ler Arquivo", command=self.Ler_Arquivo_DataFrame)
        self.button_LerArquivo.grid(row=4, column=0, padx=20, pady=(10, 20))

        # Adicione este botão à interface
        self.button_cancel_load = customtkinter.CTkButton(self, text="Cancelar Carregamento", command=self.cancel_load)
        self.button_cancel_load.grid(row=5, column=1, padx=20, pady=(10, 10))

        # Debug
        self.label_debug = customtkinter.CTkFrame(self, width=50, corner_radius=0)
        self.label_debug.grid(row=6, column=1, columnspan=2, sticky="nsew")
        self.label_debug.grid_rowconfigure(4, weight=1)

        # Debug entry
        self.entry = customtkinter.CTkLabel(self.label_debug, text="Selecione um arquivo")
        self.entry.grid(row=6, column=1)

        self.progress_bar = tkinter.ttk.Progressbar(self.label_debug, orient='horizontal', length=500, mode='determinate')
        self.progress_bar.grid(row=7, column=1, padx=20, pady=(10, 10))

        # create tabview
        self.scrollable_frame = customtkinter.CTkScrollableFrame(self, label_text="Selecione as Colunas:")
        self.scrollable_frame.grid(row=0, column=2, padx=(20, 0), pady=(20, 0), sticky="nsew")
        self.scrollable_frame.grid_columnconfigure(0, weight=1)
        self.scrollable_frame_switches = []

        # Botão para carregar as colunas selecionadas
        self.button_carregar_colunas = customtkinter.CTkButton(self, text="Carregar DataFrame", command=self.carregar_colunas_selecionadas)
        self.button_carregar_colunas.grid(row=1, column=2, padx=20, pady=10)

        # Adicionando variáveis para armazenar os valores dos switches
        self.switch_variables = []

    def change_appearance_mode_event(self, new_appearance_mode):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def change_scaling_event(self, new_scaling):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        customtkinter.set_widget_scaling(new_scaling_float)
    
    @staticmethod
    def remove_special_characters(text):
        text = ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))
        return text.upper()
    
    def determinar_delimitador(self, arquivo):
        with open(arquivo, 'r') as file:
            line = file.readline()
            delimiters = [',', ';', '\t']
            for delimiter in delimiters:
                if delimiter in line:
                    return delimiter
            return ','
    
    def on_base_dados_selected(self, value):
        self.selected_file = value  # Armazenar o arquivo selecionado ao escolher na opção
        self.entry.configure(text=f"Selecionado o arquivo {self.selected_file}")

    def on_extensao_selected(self, value):
        self.selected_extension = value  # Armazenar a extensão selecionada ao escolher na opção
        self.entry.configure(text=f"Selecionada a extensão {self.selected_extension}")

    def mostrar_colunas_disponiveis(self, df):
        columns = df.columns.tolist()  # Obtém os nomes das colunas como uma lista
        # Após a leitura bem-sucedida do arquivo CSV, cria os interruptores para as colunas
        for i, col in enumerate(columns):
            switch_var = tkinter.IntVar(value=0)  # Variável para armazenar o valor do interruptor
            switch = customtkinter.CTkSwitch(master=self.scrollable_frame, text=f"{col}", variable=switch_var)
            switch.grid(row=i, column=0, padx=10, pady=(0, 20))
            self.scrollable_frame_switches.append(switch)
            self.switch_variables.append(switch_var)

    def on_Barra_de_Carregamento(self):
        self.entry.configure(text="Lendo arquivo...")

        try:
            # Simulação de leitura do arquivo (substitua pela leitura real do arquivo)
            import time
            for i in range(100):
                time.sleep(0.1)  # Simulando uma operação demorada
                self.progress_bar['value'] = i + 1

                # Adicione as linhas abaixo para permitir o cancelamento
                if self._cancel_load:
                    self._cancel_load = False
                    self.progress_bar['value'] = 0
                    self.entry.configure(text="Carregamento cancelado.")
                    return

            self.progress_bar['value'] = 0  # Reinicia a barra de progresso

        except FileNotFoundError as e:
            self.entry.configure(text="Arquivo não encontrado.")
        except Exception as e:
            self.entry.configure(text="Erro ao carregar arquivo.")
    
    def cancel_load(self):
        # Este método será chamado quando o botão de cancelamento for pressionado
        self._cancel_load = True
        self.progress_bar['value'] = 0  # Reinicia a barra de progresso
        self.entry.configure(text="Carregamento cancelado.")

    def Ler_Arquivo_DataFrame(self):
        if self.selected_extension == ".csv":
            file_path = filedialog.askopenfilename(initialdir="/", title="Selecione um arquivo",
                                                filetypes=(("Arquivos CSV", "*.csv"), ("Todos os arquivos", "*.*")))
            if file_path:  # Verifica se um arquivo foi selecionado
                self.on_Barra_de_Carregamento()

                df_temp = pd.read_csv(file_path, delimiter=self.determinar_delimitador(file_path), low_memory=False)
                self.mostrar_colunas_disponiveis(df_temp)

                # Armazena temporariamente o DataFrame no arquivo temporário
                df_temp.to_csv(self.temp_file.name, index=False)

                self.entry.configure(text="Arquivo carregado com sucesso! Visualize as colunas disponíveis.")
            else:
                self.entry.configure(text="Nenhum arquivo selecionado.")
        else:
            self.entry.configure(text="Selecione um arquivo CSV antes de ler!")

    def carregar_colunas_selecionadas(self):
        if not os.path.exists(self.temp_file.name):
            self.entry.configure(text="Nenhum arquivo temporário encontrado. Carregue um arquivo antes de selecionar colunas.")
            return

        # Obtém as colunas selecionadas
        colunas_selecionadas = [self.scrollable_frame_switches[i].cget("text") for i, var in enumerate(self.switch_variables) if var.get()]

        if colunas_selecionadas:
            while True:  # Continue até que o usuário selecione 'No' na pergunta
                # Carrega apenas as colunas selecionadas do arquivo temporário
                df_selecionado = pd.read_csv(self.temp_file.name, usecols=colunas_selecionadas, low_memory=False)

                # Pergunta ao usuário se deseja carregar mais uma base
                resposta = tkinter.messagebox.askquestion("SAUDADOS", "Deseja carregar mais alguma base de dados?")

                # Adiciona o DataFrame atual à lista de DataFrames carregados
                self.dataframes_carregados.append(df_selecionado)

                # Obtém o nome do arquivo selecionado
                nome_arquivo = os.path.basename(self.selected_file)

                if resposta == 'yes':
                    # Cria um novo arquivo temporário para a próxima base
                    novo_temp_file = tempfile.NamedTemporaryFile(delete=False)

                    # Guarda o DataFrame no novo arquivo temporário
                    df_selecionado.to_csv(novo_temp_file.name, index=False)

                    # Limpa a interface e o arquivo temporário atual
                    self.scrollable_frame.destroy()
                    self.scrollable_frame_switches = []
                    self.switch_variables = []
                    self.temp_file.close()

                    # Atualiza o arquivo temporário atual com o novo
                    self.temp_file = novo_temp_file

                    # Reinicia a interface
                    self.init_interface()

                    self.entry.configure(text=f"Arquivo '{nome_arquivo}' carregado com sucesso! Visualize as colunas disponíveis.")
                else:
                    # Exibe todos os DataFrames carregados até agora
                    for i, df in enumerate(self.dataframes_carregados, start=1):
                        self.mostrar_dataframe_selecionado(df, f"DataFrame {i} - {nome_arquivo}")

                    break  # Sai do loop ao carregar todas as bases

        else:
            self.entry.configure(text="Selecione pelo menos uma coluna para carregar.")

    def aplicar_filtro(self, dataframe, expressao_filtro, text_widget):
        try:
            # Aplica o filtro ao DataFrame
            dataframe_filtrado = dataframe.query(expressao_filtro)

            # Atualiza o texto na janela com o DataFrame filtrado
            text_widget.delete("1.0", tkinter.END)  # Limpa o conteúdo atual do Text widget
            text_widget.insert("0.0", dataframe_filtrado.to_string())

        except Exception as e:
            # Exibe uma mensagem de erro se a expressão de filtro for inválida
            tkinter.messagebox.showerror("Erro", f"Erro ao aplicar filtro: {str(e)}")

    def mostrar_dataframe_selecionado(self, dataframe, nome_arquivo):
        # Cria uma nova janela para exibir o DataFrame
        janela_dataframe = tkinter.Toplevel(self)
        janela_dataframe.title(f"DataFrame - {nome_arquivo}")
        janela_dataframe.geometry(f"{1100}x{580}")
        janela_dataframe.resizable(width=False, height=False)

        # Adiciona widgets para cada coluna (interruptor e OptionMenu)
        for i, col in enumerate(dataframe.columns):
            switch_var = tkinter.IntVar(value=0)  # Variável para armazenar o valor do interruptor
            # switch = customtkinter.CTkSwitch(master=janela_dataframe, text=f"{col}", variable=switch_var)
            # switch.grid(row=i, column=0, padx=10, pady=(0, 20))
            # self.scrollable_frame_switches.append(switch)
            # self.switch_variables.append(switch_var)

            # Adiciona um OptionMenu para escolher os valores únicos da coluna
            valores_unicos = dataframe[col].unique()
            valores_unicos_var = tkinter.StringVar(value=valores_unicos[0])
            option_menu = customtkinter.CTkOptionMenu(janela_dataframe, values=[str(val) for val in valores_unicos], variable=valores_unicos_var)
            option_menu.grid(row=i, column=1, padx=10, pady=(0, 20))
            self.scrollable_frame_switches.append(option_menu)
            self.switch_variables.append(valores_unicos_var)

        # Adiciona um widget Entry para inserir a expressão de filtro
        entry_filtro = customtkinter.CTkEntry(janela_dataframe)
        entry_filtro.grid(row=i + 1, column=0, padx=20, pady=(10, 0))

        # Adiciona um widget Button para aplicar o filtro
        btn_aplicar_filtro = customtkinter.CTkButton(janela_dataframe, text="Aplicar Filtro",
                                                    command=lambda: self.aplicar_filtro(dataframe, entry_filtro.get(), text_widget))
        btn_aplicar_filtro.grid(row=i + 1, column=1, padx=10, pady=(10, 0))

        # Adiciona um widget Text para exibir o DataFrame
        text_widget = customtkinter.CTkTextbox(janela_dataframe, width=400)
        text_widget.grid(row=i + 2, column=0, columnspan=2, padx=20, pady=20)

        # Insere o DataFrame no widget Text
        text_widget.insert("1.0", dataframe.to_string())  # Usar "1.0" para inserir no início

if __name__ == "__main__":
    app = AppSaudados()
    app.mainloop()

KeyboardInterrupt: 